In [6]:
import lxml
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [7]:
from fake_useragent import UserAgent                
ua = UserAgent()
headers = {'User-Agent': ua.random}


In [8]:
url = "https://www.transfermarkt.es/valencia-cf/kader/verein/1049/plus/0/galerie/0?saison_id=2014"

In [9]:
plantilla15 = requests.get(url, headers=headers)


In [10]:
plantilla15 = BeautifulSoup(plantilla15.text, "html.parser")

In [11]:
temp15={"Nombre":[],
        "Posicion":[],
        "Edad":[],
        "Nacionalidad":[],
        "Valor":[]
        }

SACAR EL VALOR DEL JUGADOR EN EL EQUIPO

In [12]:
for valor in plantilla15.find_all("td", class_ = "rechts hauptlink"):
    temp15["Valor"].append(valor.text)

SACAR NOMBRE JUGADOR

In [13]:
nombres = plantilla15.find_all('img', class_='bilderrahmen-fixed lazy lazy')
for nombre in nombres:
    #jugador24 = nombre["title"]
    temp15["Nombre"].append(nombre["title"])


SACAR EDAD JUGADOR

Veo que desde la posicion 1 y cada 4 saco el TD donde esta la edad ya que ese campo no tiene ningún atributo o clase que pueda diferenciar del resto.
Saco la longitud total de el td para saber su longitud y recorrer cada campo de la edad.

In [14]:
edad = plantilla15.find_all("td",class_ = "zentriert")

longitud=(len(edad)-4) #Saco el total y en el range le pongo 97 porque es el último valor de la cadena que queremos sacar.

valores_edad = [edad[i].text.strip() for i in range(1, longitud, 4)]
temp15["Edad"]=valores_edad

SACAR LA POSICION
Como en el anterior no tenemos una etiqueta ni nada que nos indique el campo de la posicion tenemos que sacarlo
a traves de la posicion de la etiqueta.

In [15]:
posiciones = plantilla15.find_all("table",class_ = "inline-table")

longitud_pos = (len(posiciones))
for i in range(longitud_pos):
    #print(posiciones[i].find_all("td")[2].text.strip())
    temp15["Posicion"].append(posiciones[i].find_all("td")[2].text.strip())

SACAR NACIONALIDAD

saco el td con la clase, veo que los que tienen doble nacionalidad ocupan td diferentes por eso le decimos que coja todos los "img" y la primera posicion de cada td.

In [16]:
nacionalidades = plantilla15.find_all("td", class_='zentriert')
for nacion in nacionalidades:
    img = nacion.find_all("img", class_="flaggenrahmen")
    if img:
        #print(img[0].get("title"))
        temp15["Nacionalidad"].append(img[0].get("title"))

In [17]:
temporada15 =  pd.DataFrame(temp15)

In [18]:
temporada15.head(10)

,Nombre,Posicion,Edad,Nacionalidad,Valor
0,Diego Alves,Portero,30,Brasil,"8,00 mill. €"
1,Yoel Rodríguez,Portero,26,España,"2,50 mill. €"
2,Jaume Doménech,Portero,24,España,-
3,Nicolás Otamendi,Defensa central,27,Argentina,"18,00 mill. €"
4,Shkodran Mustafi,Defensa central,23,Alemania,"15,00 mill. €"
5,Lucas Orbán,Defensa central,26,Argentina,"5,00 mill. €"
6,Rúben Vezo,Defensa central,21,Portugal,"2,50 mill. €"
7,José Gayà,Lateral izquierdo,20,España,"15,00 mill. €"
8,Salva Ruiz,Lateral izquierdo,20,España,-
9,Antonio Barragán,Lateral derecho,28,España,"3,00 mill. €"


In [19]:
temporada15.Edad = temporada15.Edad.astype(int)

In [20]:
temporada15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Nombre        26 non-null     object
 1   Posicion      26 non-null     object
 2   Edad          26 non-null     int32 
 3   Nacionalidad  26 non-null     object
 4   Valor         26 non-null     object
dtypes: int32(1), object(4)
memory usage: 1.0+ KB


In [21]:
temporada15.Valor = temporada15.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\raul_\AppData\Local\Temp\ipykernel_7668\3851265430.py:1: SyntaxWarning: invalid escape sequence '\d'
  temporada15.Valor = temporada15.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)


Pasamos los valores a millones, sabemos que los valores altos que habian no corresponden al valor en millones ya que sabemos que 
la plantilla del valencia no tiene jugadores tan valiosos.

In [22]:
temporada15.loc[temporada15.Valor > 100,"Valor"] = temporada15.loc[temporada15.Valor > 100,"Valor"]/1000


In [23]:
temporada15["Valor millones"] = temporada15.Valor 

In [24]:
temporada15 = temporada15.drop(columns=["Valor"])

In [25]:
temporada15

,Nombre,Posicion,Edad,Nacionalidad,Valor millones
0,Diego Alves,Portero,30,Brasil,8.0
1,Yoel Rodríguez,Portero,26,España,2.5
2,Jaume Doménech,Portero,24,España,NaN
3,Nicolás Otamendi,Defensa central,27,Argentina,18.0
4,Shkodran Mustafi,Defensa central,23,Alemania,15.0
5,Lucas Orbán,Defensa central,26,Argentina,5.0
6,Rúben Vezo,Defensa central,21,Portugal,2.5
7,José Gayà,Lateral izquierdo,20,España,15.0
8,Salva Ruiz,Lateral izquierdo,20,España,NaN
9,Antonio Barragán,Lateral derecho,28,España,3.0


In [26]:
#temporada22["Valor millones"].fillna(temporada22["Valor millones"].min, inplace=True)
min_valor = temporada15["Valor millones"].min()
temporada15["Valor millones"].fillna(min_valor, inplace=True)

C:\Users\raul_\AppData\Local\Temp\ipykernel_7668\2550227724.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temporada15["Valor millones"].fillna(min_valor, inplace=True)


In [27]:
ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada15_14.csv"

temporada15.to_csv(ruta, index="False")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\raul_\AppData\Local\Temp\ipykernel_7668\1874498574.py:1: SyntaxWarning: invalid escape sequence '\D'
  ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada15_14.csv"
